In [2]:
import pickle
import os
import pandas as pd
from fairness import *
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt



try: 
    with open('Data.pkl', 'rb') as f:
        Data = pickle.load(f)
    with open('sens_attr_dict.pkl', 'rb') as f:
        sens_attr_dict = pickle.load(f)

except:

    Data = []
    result =[]
    for row in os.listdir('SCORES'):
        if 'DS_Store' not in row: 
            model , dataset = row.replace('score_','').rstrip('.csv').split('_',1)
            if model not in ['SVM', 'LogReg', 'LinReg']:
                y_true = np.array(pd.read_csv('DATA/' + dataset + '/test.csv')['label'])
                df = pd.read_csv('DATA/' + dataset + '/test.csv')
                score = pd.read_csv('SCORES/'+row)
                score = np.array(score[score.columns[0]])
            else:
                if dataset == 'DBLP-GoogleScholar': continue
                df_ = pd.read_csv('SCORES/'+row)
                y_true = np.array(df_['label']).reshape(-1)
                score = np.array(df_['score']).reshape(-1)
                df_ = df_[['left','right']]
                df = df_.rename(columns={'left': 'left_'+sens_dict[dataset][0], 'right': 'right_'+sens_dict[dataset][0]})
            result.append([score, y_true,model, dataset, df])

    sens_attr_dict ={}



    for i,row in enumerate(result):
        dataset  = row[-2]
        df = row[-1]
        if dataset not in list(sens_attr_dict.keys()):
            sens_attr = make_sens_vector(df , dataset, sens_dict)
            sens_attr_dict[dataset] = sens_attr
        else:
            sens_attr = sens_attr_dict[dataset]
            if sens_attr.shape[0] != score.shape[0]:
                sens_attr = make_sens_vector(df , dataset, sens_dict)

        Data.append([row[0],row[1],sens_attr,row[2],row[3]])
        

        
    with open('sens_attr_dict.pkl', 'wb') as f:
        pickle.dump(sens_attr_dict, f)
    with open('Data.pkl', 'wb') as f:
        pickle.dump(Data, f)




## introduction figures

In [4]:



MODEL = 'LogReg'
DATASET = 'Amazon-Google'


for row in Data:
    [score, y_true,sens_attr ,model,dataset] = row
    if dataset == DATASET and model == MODEL:
        score_g1 = score[sens_attr ==1]
        score_g2 = score[sens_attr ==0]
        break
    


score_g1 = score[sens_attr == 1]
score_g2 = score[sens_attr == 0]


E_opp__sens =[]
E_opp__non_sens = []
E_odds_sens =[]
E_odds__non_sens=[]
start = 0
end = 1
step = 100
range  = np.linspace(start, end, step)
for TH in range:

    y_pred = np.array([1 if score > TH else 0 for score in score])
    additional_fairness_metrics = calculate_additional_fairness_metrics2(y_true, y_pred, sens_attr)
    e_opp__sens = additional_fairness_metrics[0]['e_opp__sens'] 
    e_opp__non_sens = additional_fairness_metrics[0]['e_opp__non_sens'] 
    
    e_odds_sens = additional_fairness_metrics[0]['e_odds_sens'] 
    e_odds__non_sens = additional_fairness_metrics[0]['e_odds__non_sens']

    E_opp__sens.append(e_opp__sens)
    E_opp__non_sens.append(e_opp__non_sens)
    
    E_odds_sens.append(e_odds_sens)
    E_odds__non_sens.append(e_odds__non_sens)

fpr1, tpr1, _ = roc_curve(y_true[sens_attr ==1],  score[sens_attr ==1],drop_intermediate=False)
fpr2, tpr2, _ = roc_curve(y_true[sens_attr ==0],  score[sens_attr ==0],drop_intermediate=False)
auc1 = roc_auc_score(y_true[sens_attr ==1], score[sens_attr ==1])
auc2 = roc_auc_score(y_true[sens_attr ==0], score[sens_attr ==0])


EO_opps_dist = calc_DP_TPR(sens_attr, y_true, score) # DP_TPR


minority_col = "#FF5733"  # A shade of red
majority_col = "#33AFFF"  # A shade of blue
green_color= "#C7E9B4"


 

L = 1.5
F = 28
F_legend = 22
F_title = 32
size = (8,6)

plt.figure(figsize=size)
plt.plot(range,E_opp__sens,label ='Minority', color = minority_col)
plt.plot(range,E_opp__non_sens, label ='Majority', color = majority_col)
legend = plt.legend(fontsize = F_legend)
legend.get_frame().set_edgecolor('black')
plt.xlabel('Threshold (' + r'$\tau$'+')', fontsize =F_title)  
plt.ylabel('TPR', fontsize =F_title)  
plt.xticks(fontsize = F)
plt.yticks(fontsize = F)
plt.fill_between(range, E_opp__sens, E_opp__non_sens, color='#C0C0C0', alpha=0.3)
plt.ylim([0,1.015])
plt.xlim([0,1])
plt.tight_layout()
plt.gca().get_xticklabels()[0].set_horizontalalignment('center')
plt.gca().get_yticklabels()[0].set_verticalalignment('center')
plt.savefig('FIGURES/introduction/Intro_EQ_OPP.pdf')
plt.close()

plt.figure(figsize=size)
plt.plot(fpr1,tpr1,label ='Minority', color = minority_col, linewidth = L)
plt.plot(fpr2,tpr2, label ='Majority', color = majority_col, linewidth = L)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')  # Adding y=x line
legend =plt.legend(fontsize = F_legend)
legend.get_frame().set_edgecolor('black')
plt.xlabel('FPR', fontsize =F_title)  
plt.ylabel('TPR', fontsize =F_title)
plt.fill_between(fpr1, tpr1, color=minority_col, alpha=0.2)
plt.fill_between(fpr2, tpr2, color=majority_col, alpha=0.2)
plt.ylim([0,1.015])
plt.xlim([-0.01,1])
plt.xticks(fontsize = F)
plt.yticks(fontsize = F)
plt.gca().get_xticklabels()[0].set_horizontalalignment('center')
plt.gca().get_yticklabels()[0].set_verticalalignment('center')
plt.tight_layout()
plt.savefig('FIGURES/introduction/Intro_AUC.pdf')
plt.close()



## Before/After calibration Figures

In [8]:
color_dict = {'minority':"#FF5733",
              'majority': "#33AFFF",
              'total':'#006400'}
G_dict = {'ACC':['minority','majority'],
          'AUC':['minority','majority'],
          'Equalized opportunity':['majority','minority'],
          'Equalized odds':['minority','majority'],
          'F1-score':['minority','majority'],
          'Positive Rate':['majority','minority'],
          }


cnt =0 

df1 = []
df2 = []
df3 = []
df4 = []

for row in Data:
    [score, y_true,sens_attr ,model,dataset] = row
    score_g1 = score[sens_attr ==1]
    score_g2 = score[sens_attr ==0]
    

    bary_wass, A, bin_centers1, bin_centers2 = calc_bary2(score,sens_attr, True)

    num  = min(int(max(calc_bin(score_g1), calc_bin(score_g2))), 400)
    hist1, bin_edges1 = np.histogram(score_g1, bins=np.linspace(0, 1, num+1 ))
    hist2, bin_edges2 = np.histogram(score_g2, bins=np.linspace(0, 1, num+1 ))
    bin_centers1_ = 0.5 * (bin_edges1[:-1] + bin_edges1[1:])
    bin_centers2_ = 0.5 * (bin_edges2[:-1] + bin_edges2[1:])
    hist1 = hist1 / np.sum(hist1)
    hist2 = hist2 / np.sum(hist2)


    mapper1 = ot.da.MappingTransport(mu=1e-3, eta=1e-20, bias=False, max_iter=2000, verbose= False, metric = 'euclidean', tol = 1e-5)
    mapper1.fit(Xs=hist1.reshape(-1, 1),Xt = bary_wass.reshape(-1,1))

    mapper2 = ot.da.MappingTransport(mu=1e-3, eta=1e-20, bias=False, max_iter=2000, verbose= False, metric = 'euclidean',tol = 1e-5)
    mapper2.fit(Xs=hist2.reshape(-1, 1), Xt =  bary_wass.reshape(-1,1))

    scores_list_1_mapped = mapper1.transform(Xs=hist1.reshape(-1, 1)).ravel()
    scores_list_2_mapped = mapper2.transform(Xs=hist2.reshape(-1, 1)).ravel()


    original_hist, _ = np.histogram(score_g1, bins=len(scores_list_1_mapped), range=(min(bin_edges1), max(bin_edges1)))
    original_hist = original_hist / np.sum(original_hist)
    target_hist = scores_list_1_mapped / np.sum(scores_list_1_mapped)
    original_bin_midpoints = (np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[:-1] + np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[1:]) / 2
    target_bin_midpoints = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    cost_matrix = ot.dist(bin_centers1[:,None], target_bin_midpoints[:, None], metric='sqeuclidean')
    optimal_transport_plan = ot.emd(original_hist, target_hist, cost_matrix)
    transformed_indices = np.argmax(optimal_transport_plan, axis=1)
    transformed_data = np.interp(score_g1, bin_centers1, target_bin_midpoints[transformed_indices])


    original_hist, _ = np.histogram(score_g2, bins=len(scores_list_2_mapped), range=(min(bin_edges1), max(bin_edges1)))
    original_hist = original_hist / np.sum(original_hist)
    target_hist = scores_list_2_mapped / np.sum(scores_list_2_mapped)
    original_bin_midpoints = (np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[:-1] + np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[1:]) / 2
    target_bin_midpoints = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    cost_matrix = ot.dist(bin_centers1[:,None], target_bin_midpoints[:, None], metric='sqeuclidean')
    optimal_transport_plan = ot.emd(original_hist, target_hist, cost_matrix)
    transformed_indices = np.argmax(optimal_transport_plan, axis=1)
    transformed_data2 = np.interp(score_g2, bin_centers1, target_bin_midpoints[transformed_indices])

    map_score = np.zeros(score.shape)
    map_score[sens_attr == 1] = transformed_data 
    map_score[sens_attr == 0] = transformed_data2




    def objective(gamma, score, y_true, sens_attr,score_repair):
        score_best = score * (1-gamma) + gamma * score_repair
        Eodd_disp = calc_EO_disp(sens_attr, y_true, score_best)
        Eop_disp = calc_DP_TPR(sens_attr, y_true, score_best)
        PR_disp = calc_DP_PR(sens_attr, y_true, score_best)
        return Eodd_disp, Eop_disp, PR_disp


    func_Eodd,func_PR,func_Eop = [], [] ,[]
    for gamma in np.linspace(0, 1, 200):
        Eodd_disp, Eop_disp, PR_disp = objective(gamma, score, y_true, sens_attr, score_repair= map_score)
        func_Eodd.append([gamma, Eodd_disp ])
        func_Eop.append([gamma, Eop_disp])
        func_PR.append([gamma, PR_disp])
        


    func_Eop.sort(key= lambda x:x[1])
    func_Eodd.sort(key= lambda x:x[1])
    func_PR.sort(key= lambda x:x[1])



    cnt+=1
    print(cnt,'/',len(Data),':',MODEL+ ' '+ dataset)
        

    gamma_Eop = func_Eop[0][0]
    gamma_Eodd = func_Eodd[0][0]
    gamma_PR = func_PR[0][0]
    
    score_optimal_Eop = score * (1-gamma_Eop) + gamma_Eop * map_score
    score_optimal_Eodd = score * (1-gamma_Eodd) + gamma_Eodd * map_score
    score_optimal_PR = score * (1-gamma_PR) + gamma_PR * map_score

    plot_bef_after(score_optimal_Eop,score_optimal_Eodd,score_optimal_PR, model,dataset,sens_attr, y_true, score)



    
    df1 = do_job(df1 , score_optimal_Eop,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_Eop',gamma_Eop)
    df2 = do_job(df2 , score_optimal_Eodd,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_Eodd',gamma_Eodd)
    df3 = do_job(df3 , score_optimal_PR,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_PR',gamma_PR)



df1.to_csv('Metric_optimal_Eop.csv',index = False)
df2.to_csv('Metric_optimal_Eodd.csv',index = False)
df3.to_csv('Metric_optimal_PR.csv',index = False)


1 / 53 : HierMatcher DBLP-GoogleScholar
2 / 53 : HierMatcher Beer
3 / 53 : HierMatcher iTunes-Amazon
4 / 53 : HierMatcher Beer
5 / 53 : HierMatcher DBLP-GoogleScholar
6 / 53 : HierMatcher Walmart-Amazon
7 / 53 : HierMatcher Amazon-Google
8 / 53 : HierMatcher iTunes-Amazon


KeyboardInterrupt: 